* Overview of Pagination
* Additional Parameters and Attributes for Pagination
* Add Pagination to users API
* Validate users API with Pagination
* Process all the users data using Pagination
* Exercise and Solution - Pagination for orders

In [ ]:
# Before getting started make sure to reset the database tables
# Login to flask shell and run db.drop_all() and db.create_all()

# Generate Fake Data for users
from faker import Faker
faker = Faker()
users = []
usernames = set()
ctr = 0

while True:
    if ctr == 10000:
        break
    first_name = faker.first_name()
    last_name = faker.last_name()
    username = f'{first_name[:1]}{last_name}'.lower()
    email = f'{username}@email.com'
    if username not in usernames:
        usernames.add(username)
        user = {
            'id': '',
            'first_name': first_name,
            'last_name': last_name,
            'username': username,
            'email': email
        }
        users.append(user)
        ctr = ctr + 1

len(users)

In [ ]:
import requests
import json

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
batch_size = int(input('Enter batch size: '))

In [ ]:
for i in range(0, len(users), batch_size):
    users_chunk = users[i:i+batch_size]
    response = requests.post(
        f'{base_url}/users', 
        data={'users': json.dumps(users_chunk)}
    )
    print(response.status_code)

* Overview of Pagination

We need to paginate the results, if the API is expected to return bulk data.
1. If we do not paginate, the request might run into resource contention as the resultset size grow big.
2. Pagination will result in multiple API calls. Each call will return the number of records based on the page size.

* Additional Parameters and Attributes for Pagination

While implementing pagination, we consider additional parameters and attributes as part of the results.

1. `pageSize` - the maximum number of records each page should contain. It is both an attribute and parameter. Based on the parameter value, the maximum number of records should be returned.
2. `pageToken` - the token based on which next set of records should be fetched. It is typically the last value of primary key or unique key value in each API call.

Note: If `pageToken` is not passed, then we will fetch the first set of results based on `pageSize`. The result will contain `pageToken` only if we have the number of records equivalent to `pageSize`.

* Add Pagination to users API

Update `users()` for GET API with following pagination logic. It takes care of the following:

1. Use parameters such as `pageToken` and `pageSize`.
2. Apply `pageSize` with threshold (let's say 100 is the max for `pageSize`).
3. Ensure data is sorted and `pageToken` is computed. `pageToken` is nothing but last `id` in the result set.
4. In case the record count is less than `pageSize`, `pageToken` should not be returned.
5. Make sure to update doc string with additional parameters so that they are reflected in apidocs.

```python
@app.route('/users')
def users():
    """
    Retrieve users
    ---
    parameters:
      - name: email
        in: query
        type: string
        required: false
      - name: pageSize
        in: query
        type: integer
        required: false
      - name: pageToken
        in: query
        type: integer
        required: false
    responses:
      200:
        description: Users retrieved successfully
    """
    page_token = int(request.args.get('pageToken')) if request.args.get('pageToken') else None
    search_email = request.args.get('email', '')
    page_size = int(request.args.get('pageSize')) \
        if request.args.get('pageSize') and int(request.args.get('pageSize')) <= 100 \
        else 25

    if search_email:
        # Query the database and filter users based on the pattern match
        if page_token:
            user_recs = db.session.query(User).filter(User.email.like(f'{search_email.lower()}%')).filter(User.id > page_token).order_by(User.id).limit(page_size).all()
        else:
            user_recs = db.session.query(User).filter(User.email.like(f'{search_email.lower()}%')).order_by(User.id).limit(page_size).all()
    else:
        # Retrieve all users if no search query is provided
        if page_token:
            user_recs = db.session.query(User).filter(User.id > page_token).order_by(User.id).limit(page_size).all()
        else: 
            user_recs = db.session.query(User).order_by(User.id).limit(page_size).all()

    # below code throws error as the SQL Alchemy get response includes attributes
    # such as _sa_instance_state. 
    # The values of these attributes are not JSON Serializable
    # users = list(map(lambda rec: rec.__dict__, user_recs))
    users = []
    for user in user_recs:
        user.__dict__.pop('_sa_instance_state')
        users.append(user.__dict__)

    payload = {
        'records': users,
        'pageToken': users[-1]['id'] if len(users) == page_size else None,
        'recordCount': len(users)
    }
    return jsonify(payload)
```

* Validate users API with Pagination

1. Go to `/apidocs`
2. Test multiple scenarios (`/users` with out page size, with page size, with page size and token, etc)

You can also use `requests` to validate.

In [ ]:
import requests

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
requests.get(f'{base_url}/users').json()

In [ ]:
requests.get(f'{base_url}/users?pageSize=100').json()

In [ ]:
requests.get(f'{base_url}/users?pageToken=100&pageSize=100').json()

In [ ]:
requests.get(f'{base_url}/users?pageSize=101').json()

* Process all the users data using Pagination

In [ ]:
import requests

In [ ]:
base_url = input('Enter base url: ')

In [ ]:
users = []
pageToken = 0
pageSize = 100
while True:
    payload = requests.get(f'{base_url}/users?pageSize={pageSize}&pageToken={pageToken}').json()
    users.extend(payload['records'])
    if payload['recordCount'] < pageSize:
        break
    pageToken = payload['records'][-1]['id']

In [ ]:
users[0]

In [ ]:
len(users)

* Exercise - Pagination for orders

1. Make sure to populate orders data into the table (using POST API on orders).
2. Update `/orders` logic with pagination. Make sure it accept both `pageSize` and `pageToken` as parameters. The default `pageSize` should be 100 and the maximum `pageSize` can go up to 1000. In case, if the `pageSize` is greater than 1000, then it should return only 1000 records, otherwise the API should return the number of records equivalent to specified page size. You can use Chat GPT to generate the code by passing users code as reference.
3. Validate if pagination logic is working as expected or not. Here are some of the test case scenarios.
  * Invoke `/orders` without token or size. It should return 100 records.
  * Invoke `/orders` with page size 500 and token 500. It should return 500 records starting from order id 501.
  * Invoke `/orders` with page size 1001 and token 500. It should return 1000 records starting from 501.
4. Come up with required logic to process all the orders. Start with page token 0 and page size 1000.
5. Make sure to deploy the changes on to Remote VM using GitHub Action. Go to API Docs and validate to see if the end point is reflected or not.

* Solution - Pagination for orders

1. Make sure to populate orders data into the table (using POST API on orders).

```python
import requests
import json

base_url = input('Enter base url: ')
batch_size = int(input('Enter batch size (eg: 10000): '))

for i in range(0, len(orders), batch_size):
    orders_chunk = orders[i:i+batch_size]
    response = requests.post(
        f'{base_url}/orders', 
        data={'orders': json.dumps(orders_chunk)}
    )
    print(response.status_code)
```

2. Update `/orders` logic with pagination. Make sure it accept both `pageSize` and `pageToken` as parameters. The default `pageSize` should be 100 and the maximum `pageSize` can go up to 1000. In case, if the `pageSize` is greater than 1000, then it should return only 1000 records, otherwise the API should return the number of records equivalent to specified page size. You can use Chat GPT to generate the code by passing users code as reference.

```python
@app.route('/orders')
def orders():
    """
    Retrieve orders
    ---
    parameters:
      - name: pageSize
        in: query
        type: integer
        required: false
      - name: pageToken
        in: query
        type: integer
        required: false
    responses:
      200:
        description: Orders retrieved successfully
    """
    page_token = int(request.args.get('pageToken')) if request.args.get('pageToken') else None
    page_size = int(request.args.get('pageSize')) \
        if request.args.get('pageSize') and int(request.args.get('pageSize')) <= 1000 \
        else 1000
    
    if page_token:
        order_recs = db.session.query(Order). \
            filter(Order.order_id > page_token). \
            order_by(Order.order_id). \
            limit(page_size). \
            all()
    else:
        order_recs = db.session.query(Order). \
            order_by(Order.order_id). \
            limit(page_size). \
            all()

    orders = []
    for order in order_recs:
        order.__dict__.pop('_sa_instance_state')
        orders.append(order.__dict__)

    payload = {
        'records': orders,
        'pageToken': orders[-1]['order_id'] if len(orders) == page_size else None,
        'recordCount': len(orders)
    }

    return jsonify(payload)
```

3. Validate if pagination logic is working as expected or not. Here are some of the test case scenarios.
  * Invoke `/orders` without token or size. It should return 100 records.
  * Invoke `/orders` with page size 500 and token 500. It should return 500 records starting from order id 501.
  * Invoke `/orders` with page size 1001 and token 500. It should return 1000 records starting from 501.

```python
import requests

base_url = input('Enter base url: ')

requests.get(f'{base_url}/orders').json()
requests.get(f'{base_url}/orders?pageSize=500&pageToken=500').json()
requests.get(f'{base_url}/orders?pageSize=1001&pageToken=500').json()
```

4. Come up with required logic to process all the orders. Start with page token 0 and page size 1000.

```python
import requests

base_url = input('Enter base url: ')

orders = []
pageToken = 0
pageSize = 100
while True:
    payload = requests.get(f'{base_url}/orders?pageSize={pageSize}&pageToken={pageToken}').json()
    orders.extend(payload['records'])
    if payload['recordCount'] < pageSize:
        break
    pageToken = payload['records'][-1]['id']
```

5. Make sure to deploy the changes on to Remote VM using GitHub Action. Go to API Docs and validate to see if the end point is reflected or not.